In [1]:
# !unzip /home/aistudio/data/data78992/lcqmc.zip -d /home/aistudio/data/
!unzip /home/aistudio/data/data78992/paws-x-zh.zip -d /home/aistudio/data/
# !unzip /home/aistudio/data/data78992/bq_corpus.zip -d /home/aistudio/data/

Archive:  /home/aistudio/data/data78992/paws-x-zh.zip


   creating: /home/aistudio/data/paws-x-zh/
  inflating: /home/aistudio/data/paws-x-zh/train.tsv  


   creating: /home/aistudio/data/__MACOSX/
   creating: /home/aistudio/data/__MACOSX/paws-x-zh/
  inflating: /home/aistudio/data/__MACOSX/paws-x-zh/._train.tsv  
  inflating: /home/aistudio/data/paws-x-zh/dev.tsv  
  inflating: /home/aistudio/data/__MACOSX/paws-x-zh/._dev.tsv  
  inflating: /home/aistudio/data/paws-x-zh/License.pdf  
  inflating: /home/aistudio/data/__MACOSX/paws-x-zh/._License.pdf  
  inflating: /home/aistudio/data/paws-x-zh/test.tsv  


  inflating: /home/aistudio/data/__MACOSX/paws-x-zh/._test.tsv  
  inflating: /home/aistudio/data/__MACOSX/._paws-x-zh  


## 1. Load dataset

In [2]:
def load_dataset(fpath, num_row_skip=0):

    def read(fp):
        data = open(fp)

        for _ in range(num_row_skip):
            next(data)

        if "test" in fp:
            for line in data:
                line = line.strip().split('\t')
                yield line[0], line[1]
        else:
            for line in data:
                line = line.strip().split('\t')
                if len(line) == 3:
                    yield line[0], line[1], int(line[2])

    if isinstance(fpath, str):
        return list(read(fpath))
    elif isinstance(fpath, (list, tuple)):
        return [list(read(fp)) for fp in fpath]
    else:
        raise TypeError("Input fpath must be a str or a list/tuple of str")

In [3]:
train_set, dev_set, test_set = load_dataset(['./data/paws-x-zh/train.tsv', './data/paws-x-zh/dev.tsv', './data/paws-x-zh/test.tsv'])
# len(train_set), len(dev_set), len(test_set)
train_set = train_set + dev_set

## 2. Transform text

In [4]:
from paddlenlp.datasets import MapDataset
from paddle.io import BatchSampler, DataLoader
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.transformers import RobertaModel as SeqClfModel
from paddlenlp.transformers import RobertaTokenizer as PTMTokenizer
import numpy as np


MODEL_NAME = "roberta-wwm-ext-large"
tokenizer = PTMTokenizer.from_pretrained(MODEL_NAME)


def example_converter(example, max_seq_length, tokenizer):
    text_a, text_b, label = example
    encoded = tokenizer(text=text_a, text_pair=text_b, max_seq_len=max_seq_length)
    input_ids = encoded["input_ids"]
    token_type_ids = encoded["token_type_ids"]
    label = np.array([label], dtype="int64")
    return input_ids, token_type_ids, label


def get_trans_fn(max_seq_length=128, tokenizer=tokenizer):
    return lambda ex: example_converter(ex, max_seq_length, tokenizer)


batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    Stack(dtype="int64")
    ): fn(samples)


def create_dataloader(dataset,
                      trans_fn,
                      batchify_fn,
                      test=False,
                      batch_size=128,
                      shuffle=True,
                      sampler=BatchSampler):

    if test:
        dataset = [d + (0,) for d in dataset]

    if not isinstance(dataset, MapDataset):
        dataset = MapDataset(dataset)

    dataset.map(trans_fn)
    batch_sampler = sampler(dataset,
                            shuffle=shuffle,
                            batch_size=batch_size)

    dataloder = DataLoader(dataset,
                           batch_sampler=batch_sampler,
                           collate_fn=batchify_fn)

    return dataloder

[2022-03-29 01:32:57,059] [    INFO]

 - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/vocab.txt and saved to /home/aistudio/.paddlenlp/models/roberta-wwm-ext-large

[2022-03-29 01:32:57,062] [    INFO]

 - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/vocab.txt

  0%|          | 0/107 [00:00<?, ?it/s]

100%|██████████| 107/107 [00:00<00:00, 39015.09it/s]

In [5]:
max_seq_length = 128; batch_size = 32
trans_fn = get_trans_fn(max_seq_length)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn, batch_size=batch_size)
# dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn, batch_size=batch_size)
test_loader = create_dataloader(test_set, trans_fn, batchify_fn, shuffle=False, test=True, batch_size=batch_size)

## 3. Model building

In [6]:
from paddle import nn
import paddle


class PTM(nn.Layer):

    def __init__(self, pretrained_model, dropout=0.1, num_class=2):
        super().__init__()

        self.ptm = pretrained_model
        ptm_out_dim = self.ptm.config["hidden_size"]
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(ptm_out_dim, ptm_out_dim // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(ptm_out_dim // 2, num_class)

    def encoder(self, input_ids, token_type_ids):
        _, embd = self.ptm(input_ids, token_type_ids)
        embd = self.dropout(embd)
        return embd

    def forward(self, input_ids, token_type_ids):
        embd = self.encoder(input_ids, token_type_ids)
        hidden = self.relu(self.fc1(embd))
        logits = self.fc2(hidden)
        return logits

In [7]:
from paddlenlp.transformers import LinearDecayWithWarmup

epoch = 4
weight_decay = 0.001
warmup_proportion = 0.1
lr_scheduler = LinearDecayWithWarmup(4e-5, len(train_loader) * epoch,
                                         warmup_proportion)

def get_model(model):
    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
    optimizer = paddle.optimizer.AdamW(
    parameters=model.parameters(),
    learning_rate=lr_scheduler,
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params)

    criterion = paddle.nn.CrossEntropyLoss()

    model = paddle.Model(model)
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [8]:
ptm = SeqClfModel.from_pretrained(MODEL_NAME)
model = PTM(ptm)
model = get_model(model)

[2022-03-29 01:32:57,233] [    INFO]

 - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/roberta_chn_large.pdparams and saved to /home/aistudio/.paddlenlp/models/roberta-wwm-ext-large

[2022-03-29 01:32:57,235] [    INFO]

 - Downloading roberta_chn_large.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/roberta_large/roberta_chn_large.pdparams

  0%|          | 0/1271615 [00:00<?, ?it/s]

...

 98%|█████████▊| 1252458/1271615 [00:18<00:00, 72239.01it/s]

 99%|█████████▉| 1259710/1271615 [00:18<00:00, 70999.74it/s]

100%|█████████▉| 1266835/1271615 [00:18<00:00, 69517.79it/s]

100%|██████████| 1271615/1271615 [00:18<00:00, 67510.48it/s]

W0329 01:33:16.140797   256 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0329 01:33:16.144894   256 device_context.cc:465] device: 0, cuDNN Version: 7.6.


## 4. Model training

In [9]:
model.fit(train_loader, epochs=epoch, verbose=2, log_freq=100)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.

Epoch 1/4

step  100/1598 - loss: 0.6788 - acc: 0.5416 - 634ms/step


step  200/1598 - loss: 0.4744 - acc: 0.5670 - 632ms/step


step  300/1598 - loss: 0.3809 - acc: 0.6301 - 631ms/step


...

step 1200/1598 - loss: 0.0132 - acc: 0.9841 - 638ms/step


step 1300/1598 - loss: 0.0036 - acc: 0.9845 - 638ms/step


step 1400/1598 - loss: 0.0278 - acc: 0.9846 - 639ms/step


step 1500/1598 - loss: 0.0254 - acc: 0.9846 - 638ms/step


step 1598/1598 - loss: 0.0070 - acc: 0.9846 - 638ms/step


## 5. Prediction

In [10]:
import paddle.nn.functional as F


predictions = []
logits = model.predict(test_loader)

for batch in logits[0]:
    batch = paddle.to_tensor(batch)
    probs = F.softmax(batch, axis=1)
    preds = paddle.argmax(probs, axis=1).numpy().tolist()
    predictions.extend(preds)

Predict begin...

step  2/63 [..............................]

 - ETA: 17s - 291ms/step



step  4/63 [>.............................]

...

 - ETA: 0s - 242ms/step



step 63/63 [==============================]

 - 240ms/step          


Predict samples: 2000

In [11]:
with open('paws-x.tsv', 'w') as f:
    f.write("index\tprediction")
    for idx, p in enumerate(predictions):
        f.write(f"\n{idx}\t{p}")
    f.close()